In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.wkt import loads

# Load highway data
file_path_highway = "../data/raw/shapefile_data.csv"
df_highway = pd.read_csv(file_path_highway)
df_highway["geometry"] = df_highway["geometry"].apply(loads)
gdf_full = gpd.GeoDataFrame(df_highway, geometry="geometry")

# Filter for Prince Edward County
gdf = gdf_full[(gdf_full["CSDNAME_L"] == "Prince Edward County") | (gdf_full["CSDNAME_R"] == "Prince Edward County")]
gdf = gdf.set_crs(epsg=3347, allow_override=True).to_crs(epsg=4326)

# Load gas station data
file_path_gas = "../data/raw/ontario_gas_stations.csv"
df_gas_station = pd.read_csv(file_path_gas)

# Define bounding box for Prince Edward County
prince_edward_bounds = {
    "min_lat": 43.85,
    "max_lat": 44.10,
    "min_lon": -77.65,
    "max_lon": -76.90
}

# Filter gas stations within the bounding box
df_prince_edward = df_gas_station[
    (df_gas_station["Latitude"] >= prince_edward_bounds["min_lat"]) &
    (df_gas_station["Latitude"] <= prince_edward_bounds["max_lat"]) &
    (df_gas_station["Longitude"] >= prince_edward_bounds["min_lon"]) &
    (df_gas_station["Longitude"] <= prince_edward_bounds["max_lon"])
]

# Create a single combined map
m_combined = folium.Map(location=[44.00, -77.25], zoom_start=10)

# Add highways as GeoJSON
folium.GeoJson(gdf.to_json(), name="Highways").add_to(m_combined)

# Add gas stations as markers
for _, row in df_prince_edward.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Name']} ({row['Brand']})",
        icon=folium.Icon(color="blue", icon="gas-pump", prefix="fa")
    ).add_to(m_combined)

# Display the map
m_combined
